In [7]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [8]:
#Veri setini çağıralım
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60000,0.62700,50,1
1,1,85,66,29,0,26.60000,0.35100,31,0
2,8,183,64,0,0,23.30000,0.67200,32,1
3,1,89,66,23,94,28.10000,0.16700,21,0
4,0,137,40,35,168,43.10000,2.28800,33,1


In [9]:
#Kategorik ve Nümerik değişkenleri tespit edelim.

In [12]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 768
Variables: 9
cat_cols: 1
num_cols: 8
cat_but_car: 0
num_but_cat: 1


In [13]:
# 768 Gözlem 
# 9 Değişken
# 1 kategorik değişken
# 8 nümerik değişken
# 8 nümerik değişkenden birisi nümerik görünümlü olup aslında kategorik değişkendir.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [23]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.00000,3.84505,3.36958,0.00000,1.00000,3.00000,6.00000,17.00000
Glucose,768.00000,120.89453,31.97262,0.00000,99.00000,117.00000,140.25000,199.00000
BloodPressure,768.00000,69.10547,19.35581,0.00000,62.00000,72.00000,80.00000,122.00000
SkinThickness,768.00000,20.53646,15.95222,0.00000,0.00000,23.00000,32.00000,99.00000
Insulin,768.00000,79.79948,115.24400,0.00000,0.00000,30.50000,127.25000,846.00000
BMI,768.00000,31.99258,7.88416,0.00000,27.30000,32.00000,36.60000,67.10000
DiabetesPedigreeFunction,768.00000,0.47188,0.33133,0.07800,0.24375,0.37250,0.62625,2.42000
Age,768.00000,33.24089,11.76023,21.00000,24.00000,29.00000,41.00000,81.00000
Outcome,768.00000,0.34896,0.47695,0.00000,0.00000,0.00000,1.00000,1.00000


In [24]:
# Min ve Max değerleri arasında ki farklar göz önüne alındığında aykırı değerlerin varlığından söz edebiliriz.

# Aykırı Değer Analizi

In [15]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [16]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [20]:
for col in num_cols:
    print(col, check_outlier(df, col))

Pregnancies True
Glucose True
BloodPressure True
SkinThickness True
Insulin True
BMI True
DiabetesPedigreeFunction True
Age True


In [18]:
# Tüm değişkenler aykırı değere sahiptir.

In [ ]:
# Aykırı değerler için baskılama veya silme yöntemini tercih edebiliriz. Burada kullanacak olduğumuz model etkili olacaktır.

# Eksik Değer Analizi

In [25]:
df.isnull().any()

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [26]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [27]:
# Veri setinde eksik değer bulunmamaktadır.

In [28]:
# Eksik değer yok ancak Glikoz ve İnsulin gibi değerlerin 0 olması aslında o değerlerin eksik olduğu anlamına gelmektedir. Bu durumda bu değerleri NaN olarak kabul edip işlem yapabiliriz. Ancak bu çalışmada ağaç yöntemeleri kullanacağımız için eksik değerleri görmezden gelebiliriz.

# Aykırı Değerleri Silme

In [29]:
# Ağaç yöntemlerde aykırı değerlere dokunmaya gerek olmayabilir. Ancak biz bu çalışma da aykırı değerleri sileceğiz.

In [30]:
def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers


In [31]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60000,0.62700,50,1
1,1,85,66,29,0,26.60000,0.35100,31,0
2,8,183,64,0,0,23.30000,0.67200,32,1
3,1,89,66,23,94,28.10000,0.16700,21,0
4,0,137,40,35,168,43.10000,2.28800,33,1


In [32]:
num_cols = [col for col in num_cols if col not in "Outcome"]

In [33]:
df.shape

(768, 9)

In [34]:
for col in num_cols:
    new_df = remove_outlier(df, col)

df.shape[0] - new_df.shape[0]

9

In [37]:
new_df.shape[0]

759

# ENCODING İŞLEMLERİ

In [46]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [47]:
ohe_cols = [col for col in new_df.columns if 15 >= df[col].nunique() > 2]

In [48]:
df = one_hot_encoder(new_df, ohe_cols)

In [49]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60000,0.62700,50,1
1,1,85,66,29,0,26.60000,0.35100,31,0
2,8,183,64,0,0,23.30000,0.67200,32,1
3,1,89,66,23,94,28.10000,0.16700,21,0
4,0,137,40,35,168,43.10000,2.28800,33,1


In [50]:
df.shape[0]

759

# DEĞİŞKENLERİN STANDARTLAŞTIRILMASI

In [51]:
# değişkenlerin standartlatırılması
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.35294,0.74372,0.59016,0.35354,0.00000,0.50075,0.23442,0.64444,1.00000
1,0.05882,0.42714,0.54098,0.29293,0.00000,0.39642,0.11657,0.22222,0.00000
2,0.47059,0.91960,0.52459,0.00000,0.00000,0.34724,0.25363,0.24444,1.00000
3,0.05882,0.44724,0.54098,0.23232,0.11111,0.41878,0.03800,0.00000,0.00000
4,0.00000,0.68844,0.32787,0.35354,0.19858,0.64232,0.94364,0.26667,1.00000
